In [135]:
import os,sys
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

from vollib.black_scholes_merton.implied_volatility import *

GLOBAL SETTINGS

The price of an option depends on the risk free interest rate and the annualized dividend rate of the chosen underlying stock.

In [136]:
quote_data = "QuoteData.dat"
risk_free_interest_rate = 0.005
dividend_rate = 0.03

In [137]:
# TODO : Change calendar to a 252 day trading calendar (likely via Zipline module, would add a lot of code otherwise)
cumulative_month = {'Jan': 31, 'Feb': 57, 'Mar': 90,
                    'Apr': 120, 'May': 151, 'Jun': 181,
                    'Jul': 212, 'Aug': 243, 'Sep': 273,
                    'Oct': 304, 'Nov': 334, 'Dec': 365}

In [138]:
def get_expiration(string):
    
    string = string.split()
    expiration_year, expiration_month = string[0], string[1]
    expiration_date = (int(expiration_year) - (int(current_year) % 2000)) * 365 + cumulative_month[expiration_month] 
    return expiration_date

In [139]:
def get_strike(string):
    
    string = string.split()
    strike = float(string[2])
    return strike

In [140]:
def calculate_iv(bid, ask, strike, expiration_date, flag, direction):

    # Select which value to use for premium
    if "long" in direction:
        premium = float(bid)
    elif "short" in direction:
        premium = float(ask)
    elif "average" in direction:
        premium = (float(bid) + float(ask)) / 2.0
    else:
        print "Check code as direction parameter is neither long, short, or average"
        exit(1)
    
    time_to_expiry = (expiration_date - current_date) / 365.0
    
    # Convert to textbook shorthand notation
    P = premium
    S = underlying
    K = strike
    t = time_to_expiry
    r = risk_free_interest_rate
    q = dividend_rate

    # Fastest method to calculate implied volatility, using the vollib library
    sigma = implied_volatility(P, S, K, t, r, q, flag)   
    return max(sigma, 0.0)

In [141]:
# Opens CBOE QuoteData
try:
    data = open(quote_data)
    header1 = data.readline().split(",")
    header2 = data.readline().split(",")
    date = header2[0].split()
    data.close() 
except:
    print "Couldn't read QuoteData. Maybe the format changed?"
    exit(1)

# Parse the header information in QuotaData
ticker = header1[0].split()[0]
underlying = float(header1[1])
current_month, current_day, current_year = date[0], date[1], date[2]
current_date = cumulative_month[current_month] + int(current_day) - 30

# Prints visual information
print "Calculating implied volatilities"
print "Stock: %s @ %s$" % (ticker, underlying)
print "Date: %s %s %s (%sth day)" % (current_month, current_day, current_year, current_date)
#print ""
#print header1
#print header2
#print date
#print ""

Calculating implied volatilities
Stock: AAPL @ 109.8$
Date: Nov 04 2016 (308th day)


In [142]:
# Opens CBOE quotedata to get calls, and fills in NA values with 0.0, and duplicates it for puts
df = pd.read_csv(quote_data, sep=",", header = 2, mangle_dupe_cols = True, encoding="utf-8", engine = "c")
df2 = df

# Munge data by dropping useless columns, filling blank values and taking only options that have nonzero volume (eg that actually sell)   
keep = ["Calls", "Bid", "Ask", "Vol"]
df = df[keep]
df = df.fillna(0.0)
df = df[df["Vol"] > 0]
#print df

# Get expiration dates and strike prices
expiration_dates = df["Calls"].apply(get_expiration)
expiration_dates.name = "Expiration"

strikes = df["Calls"].apply(get_strike)
strikes.name = "Strike"

df = df.join(expiration_dates).join(strikes)

print "Calculating implied volatility for calls"

call_ivs = []
for index, row in df.iterrows():
    iv = calculate_iv(row["Bid"],
                    row["Ask"],
                    row["Strike"],
                    row["Expiration"],
                    "c", "average")
    call_ivs.append(iv)

print "Calculated implied volatility for %d calls" % len(df.index) 

Calculating implied volatility for calls
Calculated implied volatility for 1884 calls


In [143]:
# Repeat for put options data
# Notice the .1 for repeated names
keep2 = ["Puts", "Bid.1", "Ask.1", "Vol.1"]
df2 = df2[keep2]
df2 = df2.fillna(0.0)
df2 = df2[df2["Vol.1"] > 0] 

df2 = df2.join(expiration_dates).join(strikes)

print "Calculating implied volatility for puts"

put_ivs = []
for index, row in df2.iterrows():
    iv = calculate_iv(row["Bid.1"],
                    row["Ask.1"],
                    row["Strike"],
                    row["Expiration"],
                    "p", "average")
    put_ivs.append(iv)
    
print "Calculated implied volatility for %d puts" % len(df2.index)

Calculating implied volatility for puts
Calculated implied volatility for 1869 puts


In [145]:
# Output both calls and puts

df.to_csv(ticker + "_calls.csv", sep=",", encoding='utf-8')
df2.to_csv(ticker + "_puts.csv", sep=",", encoding='utf-8')
print "Done!"
%time

Done!
CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 21.9 µs
